In [36]:
pip install arxiv

Note: you may need to restart the kernel to use updated packages.


In [37]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.name

'wikipedia'

In [38]:
## webbase retriever
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

loader = WebBaseLoader("https://docs.langchain.com/")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 200)
final_documents = text_splitter.split_documents(docs)
vectordb = FAISS.from_documents(final_documents,HuggingFaceEmbeddings())
retriever = vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x139048dd0>, search_kwargs={})

In [39]:
## retriever
from langchain_classic.tools.retriever import create_retriever_tool
langsmith = create_retriever_tool(retriever,"langsmith_search",
                                       "Search for information about LangSmith. For any question about LangSmith you must use this tool")

langsmith.name

'langsmith_search'

In [40]:
# import pkgutil
# import langchain_classic

# print([m.name for m in pkgutil.iter_modules(langchain_classic.__path__) if "tool" in m.name])
# try:
#     import langchain_classic.tools as lc_tools
#     print("langchain_classic.tools submodules:", [m.name for m in pkgutil.iter_modules(lc_tools.__path__)])
# except Exception as e:
#     print("tools inspect error", e)

In [41]:
## arxiv retriever
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [42]:
tools = [wiki,arxiv,langsmith]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/pranshu/Documents/.venv/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 StructuredTool(name='langsmith_search', description='Search for information about LangSmith. For any question about LangSmith you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x138a2bd80>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x138a28e00>)]

In [52]:
from dotenv import load_dotenv
load_dotenv()
import os

os.environ['HUGGINGFACE_API_KEY'] = os.getenv("HUGGINGFACE_API_KEY")
from langchain_community.llms import Ollama

llm = Ollama(model="llama3.2", temperature=0)

In [44]:
from langchain_classic import hub
prompt = hub.pull("hwchase17/react")
prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

In [53]:
## agents
from langchain_classic.agents import create_react_agent

agent = create_react_agent(llm, tools, prompt)

In [46]:
# import langchainhub
# print(dir(langchainhub))

In [54]:
## Agent executor
from langchain_classic.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent,tools = tools,verbose = True, handle_parsing_errors=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': "wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangsmith_search(query: 'str', callbacks: 'Callbacks' = None) -> 'str | tuple[str, list[Document]]' - Search for information about LangSmith. For any question about LangSmith you must use this tool", 'tool_names': 'wikiped

In [55]:
agent_executor.invoke({"input" : "Tell me about Machine Learning"})



> Entering new AgentExecutor chain...
Thought: To answer this question, I should use the arxiv tool since it's related to a specific field of study (Machine Learning).

Action: arxiv
Action Input: 'machine learning'Published: 2023-06-07
Title: Changing Data Sources in the Age of Machine Learning for Official Statistics
Authors: Cedric De Boom, Michael Reusens
Summary: Data science has become increasingly essentiAction: arxiv
Action Input: 'machine learning'Published: 2023-06-07
Title: Changing Data Sources in the Age of Machine Learning for Official Statistics
Authors: Cedric De Boom, Michael Reusens
Summary: Data science has become increasingly essentiQuestion: Tell me about Machine Learning
Thought: To answer this question, I should use the arxiv tool since it's related to a specific field of study (Machine Learning).

Action: arxiv
Action Input: 'machine learning'Published: 2023-06-07
Title: Changing Data Sources in the Age of Machine Learning for Official Statistics
Authors: Cedr



> Entering new AgentExecutor chain...
Thought: To answer this question, I should use the arxiv tool since it's related to a specific field of study (Machine Learning).

Action: arxiv
Action Input: 'machine learning'Published: 2023-06-07
Title: Changing Data Sources in the Age of Machine Learning for Official Statistics
Authors: Cedric De Boom, Michael Reusens
Summary: Data science has become increasingly essentiAction: arxiv
Action Input: 'machine learning'Published: 2023-06-07
Title: Changing Data Sources in the Age of Machine Learning for Official Statistics
Authors: Cedric De Boom, Michael Reusens
Summary: Data science has become increasingly essentiQuestion: Tell me about Machine Learning
Thought: To answer this question, I should use the arxiv tool since it's related to a specific field of study (Machine Learning).

Action: arxiv
Action Input: 'machine learning'Published: 2023-06-07
Title: Changing Data Sources in the Age of Machine Learning for Official Statistics
Authors: Cedr

{'input': 'Tell me about Machine Learning',
 'output': 'Agent stopped due to iteration limit or time limit.'}